<a href="https://colab.research.google.com/github/lukaszplust/Projects/blob/main/B_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COS DZIAŁA (raczej nie tak jak trzeba)

In [16]:
class Record:
    def __init__(self, key, data):
        self.key = key
        self.data = data

    def __repr__(self):
        return f"Record({self.key}, {self.data})"

    def to_bytes(self):
        key_bytes = self.key.to_bytes(4, byteorder='big')
        data_bytes = self.data.encode()
        length_bytes = len(data_bytes).to_bytes(4, byteorder='big')
        return key_bytes + length_bytes + data_bytes

    @staticmethod
    def from_bytes(byte_data):
        key = int.from_bytes(byte_data[:4], byteorder='big')
        length = int.from_bytes(byte_data[4:8], byteorder='big')
        data = byte_data[8:8+length].decode()
        return Record(key, data)

class ReadBuffer:
    def __init__(self, path, buffer_size=4):
        self.path = path
        self.buffer_size = buffer_size
        self.buffer = []
        self.file = open(self.path, "rb")
        self.position = 0
        self.disk_reads_count = 0

    def read_next(self):
        if self.position >= len(self.buffer):
            self._fill_buffer()
        if not self.buffer:
            return None
        record = self.buffer[self.position]
        self.position += 1
        return record

    def _fill_buffer(self):
        self.buffer = []
        self.position = 0
        for _ in range(self.buffer_size):
            bytes_record = self.file.read(8)
            if not bytes_record:
                break
            length = int.from_bytes(bytes_record[4:8], byteorder='big')
            data = self.file.read(length)
            self.buffer.append(Record.from_bytes(bytes_record + data))
        self.disk_reads_count += 1

    def close(self):
        self.file.close()

class WriteBuffer:
    def __init__(self, path, buffer_size=4):
        self.path = path
        self.buffer_size = buffer_size
        self.buffer = []
        self.disk_writes_count = 0

    def write(self, record):
        self.buffer.append(record)
        if len(self.buffer) >= self.buffer_size:
            self.flush()

    def flush(self):
        with open(self.path, "ab") as file:
            for record in self.buffer:
                file.write(record.to_bytes())
        self.disk_writes_count += 1
        self.buffer = []

    def close(self):
        if self.buffer:
            self.flush()

class BTreeNode:
    def __init__(self, t, leaf=False):
        self.t = t  # Minimalny stopień (t)
        self.leaf = leaf  # Czy jest liściem
        self.keys = []  # Lista kluczy
        self.children = []  # Lista dzieci (węzłów)

    def __repr__(self):
        return f'BTreeNode(keys={self.keys}, leaf={self.leaf})'

    def find_key(self, k):
        idx = 0
        while idx < len(self.keys) and self.keys[idx] < k:
            idx += 1
        return idx

    def insert_non_full(self, k):
        i = len(self.keys) - 1
        if self.leaf:
            self.keys.append(None)
            while i >= 0 and self.keys[i] > k:
                self.keys[i + 1] = self.keys[i]
                i -= 1
            self.keys[i + 1] = k
        else:
            while i >= 0 and self.keys[i] > k:
                i -= 1
            i += 1
            if len(self.children[i].keys) == 2 * self.t - 1:
                self.split_child(i)
                if self.keys[i] < k:
                    i += 1
            self.children[i].insert_non_full(k)

    def split_child(self, i):
        t = self.t
        y = self.children[i]
        z = BTreeNode(t, y.leaf)
        self.children.insert(i + 1, z)
        self.keys.insert(i, y.keys[t - 1])
        z.keys = y.keys[t:(2 * t) - 1]
        y.keys = y.keys[0:t - 1]
        if not y.leaf:
            z.children = y.children[t:(2 * t)]
            y.children = y.children[0:t - 1]

class BTree:
    def __init__(self, t):
        self.root = BTreeNode(t, leaf=True)
        self.t = t
        self.read_operations = 0
        self.write_operations = 0

    def traverse(self):
        self._traverse(self.root)
        print()

    def _traverse(self, node):
        for i in range(len(node.keys)):
            if not node.leaf:
                self._traverse(node.children[i])
            print(node.keys[i], end=' ')
        if not node.leaf:
            self._traverse(node.children[len(node.keys)])

    def search(self, k):
        self.read_operations += 1  # Increment read operation
        return self._search(self.root, k)

    def _search(self, node, k):
        i = node.find_key(k)

        if i < len(node.keys) and node.keys[i] == k:
            return (node.keys[i], node.children[i] if not node.leaf else None)

        if node.leaf:
            return None

        return self._search(node.children[i], k)

    def insert(self, k):
        if self.search(k) is not None:
            print(f'Already exists: {k}')
            return

        root = self.root
        if len(root.keys) == (2 * self.t) - 1:
            temp = BTreeNode(self.t)
            self.root = temp
            temp.children.insert(0, root)
            self._split_child(temp, 0)
            self._insert_non_full(temp, k)
        else:
            self._insert_non_full(root, k)

        self.write_operations += 1  # Increment write operation

    def _insert_non_full(self, node, k):
        node.insert_non_full(k)
        self.write_operations += 1  # Increment write operation

    def _split_child(self, node, i):
        node.split_child(i)
        self.write_operations += 1  # Increment write operation

    def reorganize(self):
        with open(self.file, "r") as f:
            lines = f.readlines()
        lines = sorted(lines)
        with open(self.file, "w") as f:
            for line in lines:
                f.write(line)

    def print_structure(self, node=None, indent=""):
        if node is None:
            node = self.root
        print(indent + str(node.keys))
        if not node.leaf:
            for child in node.children:
                self.print_structure(child, indent + "  ")

def process_commands(btree, commands):
    for command in commands:
        parts = command.strip().split()
        if not parts:
            continue
        if parts[0] == 'insert':
            key = int(parts[1])
            btree.insert(key)
            print(f"Inserted {key}.")
            btree.print_structure()
        elif parts[0] == 'find':
            key = int(parts[1])
            found = btree.search(key)
            print(f"Key {key} {'found' if found else 'not found'}.")
        elif parts[0] == 'traverse':
            btree.traverse()
        elif parts[0] == 'stats':
            print(f"Read operations: {btree.read_operations}, Write operations: {btree.write_operations}")
        elif parts[0] == 'reorganize':
            btree.reorganize()
            print("Reorganized the B-tree.")
            btree.print_structure()
        else:
            print(f"Unknown command: {command}")

def read_commands_from_file(filename):
    with open(filename, 'r') as file:
        commands = file.readlines()
    return commands

def main():
    btree = BTree(2)
    mode = input("Enter 'file' to read commands from a file or 'interactive' for interactive mode: ")
    if mode == 'file':
        filename = input("Enter the filename: ")
        commands = read_commands_from_file(filename)
        process_commands(btree, commands)
    elif mode == 'interactive':
        while True:
            command = input("Enter command: ")
            if command == 'exit':
                break
            process_commands(btree, [command])

if __name__ == "__main__":
    main()

Enter 'file' to read commands from a file or 'interactive' for interactive mode: interactive
Enter command: insert 10
Inserted 10.
[10]
Enter command: insert 20
Inserted 20.
[10, 20]
Enter command: insert 5
Inserted 5.
[5, 10, 20]
Enter command: insert 6
Inserted 6.
[10]
  [5, 6]
  [20]
Enter command: insert 12
Inserted 12.
[10]
  [5, 6]
  [12, 20]
Enter command: insert 30
Inserted 30.
[10]
  [5, 6]
  [12, 20, 30]
Enter command: insert 7
Inserted 7.
[10]
  [5, 6, 7]
  [12, 20, 30]
Enter command: insert 17
Inserted 17.
[10, 20]
  [5, 6, 7]
  [12, 17]
  [30]
Enter command: reorganize


AttributeError: 'BTree' object has no attribute 'file'

Nowe moze działa (RACZEJ DZIAŁA PO WERYFIKACJI JUZ)

NIE WYKORZYSTUJE READBUFFER I WRITEBUFFER

In [21]:
class Record:
    def __init__(self, key, data):
        self.key = key
        self.data = data

    def __repr__(self):
        return f"Record({self.key}, {self.data})"

    def to_bytes(self):
        key_bytes = self.key.to_bytes(4, byteorder='big')
        data_bytes = self.data.encode()
        length_bytes = len(data_bytes).to_bytes(4, byteorder='big')
        return key_bytes + length_bytes + data_bytes

    @staticmethod
    def from_bytes(byte_data):
        key = int.from_bytes(byte_data[:4], byteorder='big')
        length = int.from_bytes(byte_data[4:8], byteorder='big')
        data = byte_data[8:8+length].decode()
        return Record(key, data)

class ReadBuffer:
    def __init__(self, path, buffer_size=4):
        self.path = path
        self.buffer_size = buffer_size
        self.buffer = []
        self.file = open(self.path, "rb")
        self.position = 0
        self.disk_reads_count = 0

    def read_next(self):
        if self.position >= len(self.buffer):
            self._fill_buffer()
        if not self.buffer:
            return None
        record = self.buffer[self.position]
        self.position += 1
        return record

    def _fill_buffer(self):
        self.buffer = []
        self.position = 0
        for _ in range(self.buffer_size):
            bytes_record = self.file.read(8)
            if not bytes_record:
                break
            length = int.from_bytes(bytes_record[4:8], byteorder='big')
            data = self.file.read(length)
            self.buffer.append(Record.from_bytes(bytes_record + data))
        self.disk_reads_count += 1

    def close(self):
        self.file.close()

class WriteBuffer:
    def __init__(self, path, buffer_size=4):
        self.path = path
        self.buffer_size = buffer_size
        self.buffer = []
        self.disk_writes_count = 0

    def write(self, record):
        self.buffer.append(record)
        if len(self.buffer) >= self.buffer_size:
            self.flush()

    def flush(self):
        with open(self.path, "ab") as file:
            for record in self.buffer:
                file.write(record.to_bytes())
        self.disk_writes_count += 1
        self.buffer = []

    def close(self):
        if self.buffer:
            self.flush()

class BTreeNode:
    def __init__(self, t, leaf=False):
        self.t = t  # Minimalny stopień (t)
        self.leaf = leaf  # Czy jest liściem
        self.keys = []  # Lista kluczy
        self.children = []  # Lista dzieci (węzłów)

    def __repr__(self):
        return f'BTreeNode(keys={self.keys}, leaf={self.leaf})'

    def find_key(self, k):
        idx = 0
        while idx < len(self.keys) and self.keys[idx] < k:
            idx += 1
        return idx

    def insert_non_full(self, k):
        i = len(self.keys) - 1
        if self.leaf:
            self.keys.append(None)
            while i >= 0 and self.keys[i] > k:
                self.keys[i + 1] = self.keys[i]
                i -= 1
            self.keys[i + 1] = k
        else:
            while i >= 0 and self.keys[i] > k:
                i -= 1
            i += 1
            if len(self.children[i].keys) == 2 * self.t - 1:
                self.split_child(i)
                if self.keys[i] < k:
                    i += 1
            self.children[i].insert_non_full(k)

    def split_child(self, i):
        t = self.t
        y = self.children[i]
        z = BTreeNode(t, y.leaf)
        self.children.insert(i + 1, z)
        self.keys.insert(i, y.keys[t - 1])
        z.keys = y.keys[t:(2 * t) - 1]
        y.keys = y.keys[0:t - 1]
        if not y.leaf:
            z.children = y.children[t:(2 * t)]
            y.children = y.children[0:t - 1]

class BTree:
    def __init__(self, t):
        self.root = BTreeNode(t, leaf=True)
        self.t = t
        self.read_operations = 0
        self.write_operations = 0

    def traverse(self):
        self._traverse(self.root)
        print()

    def _traverse(self, node):
        for i in range(len(node.keys)):
            if not node.leaf:
                self._traverse(node.children[i])
            print(node.keys[i], end=' ')
        if not node.leaf:
            self._traverse(node.children[len(node.keys)])

    def search(self, k):
        self.read_operations += 1  # Increment read operation
        return self._search(self.root, k)

    # s(adres strony korzenia) = node
    #
    def _search(self, node, k):
        # rownowazne z (poszukaj x na tej stronie)
        # wykonuje wyszukanie klucza w bieżącym węźle
        # find_key znajduje odpowiedni indeks dla klucza k w bieżącym węźle
        i = node.find_key(k)
        # (jezeli znaleziono xi=x, to RETURN (xi, ai))
        # jezeli klucz k znajduje sie w bieżącym węźle, zwaracny jest (ne.keys[i],..)
        # ... to referencje do dzieci, jesli węzeł nie jest liściem
        if i < len(node.keys) and node.keys[i] == k:
            return (node.keys[i], node.children[i] if not node.leaf else None)

        if node.leaf:
            return None
        # (Niech s = pi, gdzie xi < x < xi+1. Idź do kroku 2)
        # jeżeli klucz nie znajduje się w bieżącym węźle, przechodzi do
        # odpowiedniego dziecka na podstawie porównania kluczy
        return self._search(node.children[i], k)


    # ALGORYTM WYSZUKIWANIA
    def insert(self, k):
        # 1. Wyszukaj x wg algorytmu wyszukiwania klucza (z pdf)

        # metoda search sprawdza, czy klucz już istnieje w drzewie
        if self.search(k) is not None:
            # 2. jeżeli znaleziony, to RETURN (Already_Exists) (z pdf)
            print(f'Already exists: {k}')
            return
        # 3. Sprawdź na bieżącej stronie, czy m < 2d.
        # 3. Jeżeli TAK, to wstaw parę (x,a) na tę stronę i RETURN (OK).
        # Jeżeli korzeń jest pełny (ma maksymalną liczbę kluczy),
        # to tworzony jest nowy węzeł
        root = self.root

        if len(root.keys) == (2 * self.t) - 1:
            temp = BTreeNode(self.t)
            self.root = temp
            temp.children.insert(0, root)
            self._split_child(temp, 0)
            self._insert_non_full(temp, k)
        # Jeśli węzeł nie jest pełny, klucz jest dodawany bezpośrednio
        else:
            self._insert_non_full(root, k)

        self.write_operations += 1  # Increment write operation


    # 5. Spróbuj wykonać kompensację (opis poniżej).

    # Kompensacja oznacza próbę dodania klucza
    # do bieżącego węzła, jeśli nie jest on pełny,
    def _insert_non_full(self, node, k):
        node.insert_non_full(k)
        self.write_operations += 1  # Increment write operation

    # 4. / PRZEPEŁNIENIE! /
    # obsługuję przepełnienie węzłów za pomocą metody _split_child,
    # która dzieli pełny węzeł na dwa mniejsze węzły

    def _split_child(self, node, i):
        node.split_child(i)
        self.write_operations += 1  # Increment write operation

    def reorganize(self):
        all_keys = self._get_all_keys(self.root)
        all_keys.sort()
        self.root = BTreeNode(self.t, leaf=True)
        self._bulk_load(all_keys)

    def _get_all_keys(self, node):
        keys = []
        if node.leaf:
            keys.extend(node.keys)
        else:
            for i in range(len(node.keys)):
                keys.extend(self._get_all_keys(node.children[i]))
                keys.append(node.keys[i])
            keys.extend(self._get_all_keys(node.children[len(node.keys)]))
        return keys

    def _bulk_load(self, keys):
        for key in keys:
            self.insert(key)

    def print_structure(self):
        self._print_structure(self.root, 0)

    def _print_structure(self, node, level):
        print("Level", level, ":", node.keys)
        if not node.leaf:
            for child in node.children:
                self._print_structure(child, level + 1)

def process_commands(btree, commands):
    for command in commands:
        parts = command.strip().split()
        if not parts:
            continue
        if parts[0] == 'insert':
            key = int(parts[1])
            btree.insert(key)
            print(f"Inserted {key}.")
            btree.print_structure()
        elif parts[0] == 'find':
            key = int(parts[1])
            found = btree.search(key)
            print(f"Key {key} {'found' if found else 'not found'}.")
        elif parts[0] == 'traverse':
            btree.traverse()
        elif parts[0] == 'stats':
            print(f"Read operations: {btree.read_operations}, Write operations: {btree.write_operations}")
        elif parts[0] == 'reorganize':
            btree.reorganize()
            print("Reorganized the B-tree.")
            btree.print_structure()
        else:
            print(f"Unknown command: {command}")

def main():
    btree = BTree(2)
    buffer_size = 32
    mode = input("Enter 'file' to read commands from a file or 'interactive' for interactive mode: ")
    if mode == 'file':
        filename = input("Enter the filename: ")
        commands = process_commands(filename)
        process_commands(btree, commands)
    elif mode == 'interactive':
        while True:
            command = input("Enter command: ")
            if command == 'exit':
                break
            process_commands(btree, [command])

if __name__ == "__main__":
    main()

Enter 'file' to read commands from a file or 'interactive' for interactive mode: interactive
Enter command: insert 10
Inserted 10.
Level 0 : [10]
Enter command: insert 15
Inserted 15.
Level 0 : [10, 15]
Enter command: insert 6
Inserted 6.
Level 0 : [6, 10, 15]
Enter command: insert 8
Inserted 8.
Level 0 : [10]
Level 1 : [6, 8]
Level 1 : [15]
Enter command: find 6
Key 6 found.
Enter command: find 7
Key 7 not found.
Enter command: traverse
6 8 10 15 
Enter command: reorganize
Reorganized the B-tree.
Level 0 : [8]
Level 1 : [6]
Level 1 : [10, 15]
Enter command: exit
